In [1]:
# !pip install statsmodels

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os

In [2]:
pd.options.display.max_columns = None

In [3]:
pd.options.display.max_rows = 30

## Import data files

In [4]:
pwd

'D:\\data science\\CampusX\\credit_worthiness_'

In [5]:
ls

 Volume in drive D is New Volume
 Volume Serial Number is 6A78-1BA5

 Directory of D:\data science\CampusX\credit_worthiness_

18-08-2024  19:03    <DIR>          .
18-08-2024  19:01    <DIR>          ..
18-08-2024  19:03    <DIR>          .ipynb_checkpoints
18-08-2024  18:56            63,397 _01_CRM_campusX.ipynb
28-07-2024  11:05         5,382,959 case_study1.xlsx
28-07-2024  11:05        13,268,022 case_study2.xlsx
28-07-2024  11:05            12,919 Features_Target_Description.xlsx
28-07-2024  11:07    <DIR>          Material_Session 04
28-07-2024  11:06             3,181 Python Code.txt
18-08-2024  19:01                20 README.md
14-08-2024  08:00    <DIR>          session 3
               6 File(s)     18,730,498 bytes
               5 Dir(s)  51,201,110,016 bytes free


In [6]:
a1 = pd.read_excel('case_study1.xlsx') # company data
a2 = pd.read_excel('case_study2.xlsx') # bureau data

KeyboardInterrupt: 

In [ ]:
a1.shape, a2.shape

In [ ]:
a1.head()

In [ ]:
a2.head()

In [ ]:
a1.info()

In [ ]:
a2.info()

In [ ]:
df1 = a1.copy()
df2 = a2.copy()

## EDA

### Checking for null values

In [ ]:
df1.isnull().sum()

In [ ]:
df2.isnull().sum()

In [ ]:
df1.duplicated().sum()

In [ ]:
df2.duplicated().sum()

In [ ]:
len(df1['PROSPECTID'].unique())

### checking for percent columns- min/ max values, any outlier or wrong data availability 

* filtered_columns = df1.filter(regex=f'(?i){"tl"}') # this is case insensitive
* filtered_columns

In [ ]:
pct_columns = df1.filter(like='pct')

In [ ]:
pct_columns

In [ ]:
pct_columns['pct_tl_open_L6M'].describe()

In [ ]:
pct_columns['pct_tl_closed_L6M'].describe()

In [ ]:
pct_columns['pct_active_tl'].describe()

In [ ]:
pct_columns['pct_closed_tl'].describe()

In [ ]:
pct_columns['pct_tl_open_L12M'].describe()

In [ ]:
pct_columns['pct_tl_closed_L12M'].describe()

### identifying columns where null value is represented as 9999

In [ ]:
# identify all columns which has -99999 as a value
# and find percent of -99999 in any column wrt total column values

In [ ]:
for i in df1.columns:
    if df1.loc[df1[i] == -99999].shape[0] > 10000:
        print("Column: "+str(df1[i]))